In [1]:
WOS_CREDENTIALS = {
    "url": "https://zen-cpd-zen.cpd-outcomes-00-57d48117af6edd64ea27fa1142b91b0b-0000.us-south.containers.appdomain.cloud/",
    "username": "admin",
    "password": "********",
    "version": "3.5"
}

In [2]:
!pip install ibm-cloud-sdk-core --no-cache | tail -n 1
!pip install --upgrade ibm-watson-machine-learning --user | tail -n 1

In [3]:
import os
token = os.environ['USER_ACCESS_TOKEN']

WML_CREDENTIALS = {
   "token" : token,
   "instance_id" : "wml_local",
   "url": WOS_CREDENTIALS["url"],
   "version": "3.5"
}

In [4]:
import json
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

'1.0.45'

In [5]:
wml_client.spaces.list(limit=10)

------------------------------------  -----------------------------------------------------------------------  ------------------------
ID                                    NAME                                                                     CREATED
5da7339f-57a1-4f2b-92cb-d5eaf5aa18ec  attrition-demo-production                                                2021-04-07T16:29:18.352Z
c21aa2de-2354-41e4-9e62-1b0d2ee56d85  attrition-preprod                                                        2021-04-07T04:02:18.046Z
21844281-0655-4de1-af12-700ed36a5d1e  attrition-production                                                     2021-03-29T18:10:56.948Z
20213e0d-102b-4113-a317-8a5c2d8f2c01  openscale-express-path-preprod-00000000-0000-0000-0000-1616425950709603  2021-03-22T15:21:26.721Z
93d7666e-7df9-4cbb-af65-6eb93838e644  openscale-express-path-00000000-0000-0000-0000-1616425950709603          2021-03-22T15:21:09.146Z
------------------------------------  ---------------------------

In [6]:
wml_client.set.default_space('21844281-0655-4de1-af12-700ed36a5d1e')

'SUCCESS'

In [7]:
wml_client.deployments.list()

------------------------------------  -------------------------------------------------  -----  ------------------------
GUID                                  NAME                                               STATE  CREATED
7ab64005-2620-43f8-b127-2d679d9f5315  WOS-INTERNAL-4f6d61bf-53ee-4ec1-aef6-c0a08af18f0b  ready  2021-04-07T04:41:45.865Z
6b8d6961-16d2-4c1d-9389-be2c300e8bb7  predictive attrition - 2018                        ready  2021-03-29T18:12:34.965Z
------------------------------------  -------------------------------------------------  -----  ------------------------


In [8]:
deployment_guid = '6b8d6961-16d2-4c1d-9389-be2c300e8bb7'

In [9]:

import pandas as pd

df_data_1 = pd.read_csv('/project_data/data_asset/autoai_records_2020.csv')
df_data_1.head()


POSITION_CODE  DEPARTMENT_CODE  DAYS_WITH_COMPANY  COMMUTE_TIME  \
0           5700              390                940            64   
1           3800              300               1730             6   
2           5400              390               1573            12   
3           2000              320               1640            15   
4           4500              360               2065            30   

   AGE_BEGIN_PERIOD  GENDER_CODE  ATTRITION  PERIOD_TOTAL_DAYS  \
0                48            0          0                330   
1                37            1          0                330   
2                40            0          0                330   
3                45            1          0                330   
4                49            0          0                330   

   STARTING_SALARY  ENDING_SALARY  ...  VACATION_DAYS_TAKEN  SICK_DAYS_TAKEN  \
0         52230.77       54961.54  ...                   10              7.5   
1         40384.62       42846.15  ...                   11              7.5   
2         67653.85       67653.85  ...                   15             11.0   
3         60000.00       63500.00  ...                   15             10.0   
4        109230.77      116538.46  ...                   15              9.0   

   PROMOTIONS  NB_MANAGERS  DAYS_IN_POSITION  DAYS_SINCE_LAST_RAISE  \
0           0            1               940                    210   
1           0            1              1730                    270   
2           0            1              1060                    330   
3           0            1              1640                    180   
4           0            1              2065                    240   

   RANKING_CODE  OVERTIME  DBLOVERTIME  TRAVEL  
0             4       0.0          0.0       0  
1             4       0.0          0.0       0  
2             4       0.0          0.0       0  
3             2       0.0          0.0       0  
4             2       0.0          0.0       0  

[5 rows x 23 columns]

In [10]:
feed_data = df_data_1.drop(['ATTRITION'], axis=1).sample(n=100)

In [11]:
feed_data.head()

POSITION_CODE  DEPARTMENT_CODE  DAYS_WITH_COMPANY  COMMUTE_TIME  \
85            5500              390                635            18   
65            2300              320               1104             9   
52            5000              370                240            26   
163           5700              390                961            64   
114           4400              340               1975             0   

     AGE_BEGIN_PERIOD  GENDER_CODE  PERIOD_TOTAL_DAYS  STARTING_SALARY  \
85                 24            0                330         27500.00   
65                 37            1                310         58807.69   
52                 37            0                240         33000.00   
163                48            0                330         52230.77   
114                30            0                180         35961.54   

     ENDING_SALARY  NB_INCREASES  ...  VACATION_DAYS_TAKEN  SICK_DAYS_TAKEN  \
85        29615.38             1  ...                   18             11.0   
65        71500.00             1  ...                   28              4.0   
52        33000.00             0  ...                    0             10.5   
163       54961.54             1  ...                   10              7.5   
114       38076.92             1  ...                    5              0.0   

     PROMOTIONS  NB_MANAGERS  DAYS_IN_POSITION  DAYS_SINCE_LAST_RAISE  \
85            0            1               635                    270   
65            0            1               240                    240   
52            0            1               240                    240   
163           0            1               961                    210   
114           0            1              1975                    150   

     RANKING_CODE  OVERTIME  DBLOVERTIME  TRAVEL  
85              3      0.00         0.00       0  
65              3      0.00         0.00       0  
52              3      0.00         0.00       0  
163             4      0.00         0.00       0  
114             2     70.25        26.23       0  

[5 rows x 22 columns]

In [12]:
fields = feed_data.columns.tolist()
values = feed_data.values.tolist()

payload_scoring = {"input_data": [{"fields": fields, "values": values}]}  

In [13]:
predictions = wml_client.deployments.score(deployment_guid, payload_scoring)
predictions

{'predictions': [{'fields': ['prediction', 'probability'],
   'values': [[0, [0.973633372484963, 0.026366627515037072]],
    [0, [0.9993907903429441, 0.000609209657055891]],
    [0, [0.8229838429362477, 0.17701615706375223]],
    [1, [0.012350463914492682, 0.9876495360855073]],
    [1, [0.36552360992797917, 0.6344763900720208]],
    [0, [0.5922287440611025, 0.40777125593889746]],
    [0, [0.6616556503875529, 0.3383443496124471]],
    [1, [0.0528632360291762, 0.9471367639708238]],
    [0, [0.8112756417518178, 0.18872435824818226]],
    [1, [0.07367712416357053, 0.9263228758364295]],
    [1, [0.02069593969016892, 0.9793040603098311]],
    [1, [0.3743332220770922, 0.6256667779229078]],
    [1, [0.0013579576455438058, 0.9986420423544562]],
    [0, [0.9180213934401914, 0.08197860655980863]],
    [0, [0.6362245069189325, 0.36377549308106744]],
    [0, [0.9273971642019396, 0.0726028357980604]],
    [0, [0.7712970494717075, 0.22870295052829254]],
    [0, [0.9523736740713714, 0.0476263259286286